# Improve layout - 2nd LLM evaluates the 1st LLM

In [57]:
from pathlib import Path
cwd = Path.cwd()

In [68]:
chakra_docs = """Chakra UI is an accessible component system for building products with speed.\

## Documentation Sets

- [Complete documentation](chakra-v3-docs-borr8e9xb-chakra-ui.vercel.app/llms-full.txt): \
The complete Chakra UI v3 documentation including all components, styling and theming
- [Components](chakra-v3-docs-borr8e9xb-chakra-ui.vercel.app/llms-components.txt): \
Documentation for all components in Chakra UI v3.
- [Charts](chakra-v3-docs-borr8e9xb-chakra-ui.vercel.app/llms-charts.txt): \
Documentation for the charts in Chakra UI v3.
- [Styling](chakra-v3-docs-borr8e9xb-chakra-ui.vercel.app/llms-styling.txt): \
Documentation for the styling system in Chakra UI v3.
- [Theming](chakra-v3-docs-borr8e9xb-chakra-ui.vercel.app/llms-theming.txt): \
Documentation for theming Chakra UI v3.
- [Migrating to v3](chakra-v3-docs-borr8e9xb-chakra-ui.vercel.app/llms-v3-migration.txt): \
Documentation for migrating to Chakra UI v3.

## Notes

- The complete documentation includes all content from the official documentation
- Package-specific documentation files contain only the content relevant to that package"""

### Files presented to the 1st LLM

In [23]:
file1 = """
// App.jsx

import { Container, Stack, Text, Box } from "@chakra-ui/react"
import Navbar from "./components/ui/Navbar"
import TransactionGrid from "./components/ui/TransactionGrid"
import { useState } from "react"

export const BASE_URL = "http://127.0.0.1:5000/api"

function App() {
    
    const [transactions, setTransactions] = useState([])
    
    return (

            <Stack minH="100vh" gap={1}>
                <Navbar setUsers={setTransactions} />
                <Container maxW="100%" height="90vh" bg={"gray"}>
                    <Text 
                        fontSize={{ base: "3xl", md: "25" }}
                        fontWeight={"bold"}
                        letterSpacing={"2px"}
                        textAlign={"center"}
                        mb={8}
                        bg={"red"}
                        my={2}
                    >
                        <Text as={"span"} >Selected records:</Text>
                    </Text>
                    <TransactionGrid transactions={transactions} setTransactions={setTransactions} />
                </Container>
            </Stack>
    )
}

export default App
"""

In [24]:
file2 = """
// Navbar.jsx

import { Box, Flex, Text } from "@chakra-ui/react"
//import CreateUserModal from "./CreateUserModal"


function Navbar({ setTransactions }) {
    return (
        <Box px={4} bg={"gray"}>
            <Flex h="16" alignItems={"center"} justifyContent={"space-between"}>

                {/* Left side */}
                <Flex alignItems={"center"} justifyContent={"space-between"} gap={6} display={{base:"none", sm:"flex"}}>
                    <Text fontSize={"20px"}>Transactions</Text>
                    <Text fontSize={"20px"}>Categories</Text>
                </Flex>

                {/* Right side */}
                <Flex gap={6} alignItems={"center"}>
                    <Text fontSize={"20px"} fontWeight={500} display={{base: "none", md: "block"}}>
                        BFFship
                    </Text>
                </Flex>
            
            </Flex>
        </Box>
        
    )
}

export default Navbar
"""

In [25]:
file3 = """
// TransactionGrid.jsx

import { Grid, Flex, Spinner, Text, VStack } from "@chakra-ui/react";
import TransactionCard from "./TransactionCard";
import { useEffect, useState } from "react"; 
import { BASE_URL } from "../../App";

const TransactionGrid = ({ transactions, setTransactions }) => {
    
    // Initializes a state variable isLoading to true. 
    // This is used to track whether the data is currently being fetched. 
    // It will likely be used to display a loading indicator to the user.
    const [isLoading, setIsLoading] = useState(true); 
    
    // useEffect(() => { ... }, [setUsers]);: This Hook performs the data fetching logic.
    // The effect function (the first argument () => { ... }) contains the code that will run as a side effect.
    // The dependency array (the second argument [setUsers]) tells React when to re-run the effect. 
    // In this case, the effect will re-run only if the setUsers function reference changes. 
    // However, setter functions from useState are guaranteed to have a stable identity across renders, 
    // so this dependency is likely not the intended behavior. 
    // A more appropriate dependency array would likely be [] if the data should only be fetched once when the 
    // component mounts, or some other variable that indicates a need to refetch data.
    useEffect(() => {
        // First argument is a function that will run after the component mounts
        const getTransactions = async () => {
            try {
                console.log("***************");
                // Sends a GET request to the specified API endpoint. 
                // await pauses execution until the promise resolves.
                const res = await fetch(BASE_URL + "/transactions");
                const data = await res.json();
                if(!res.ok) { throw new Error(data.error); }
                
                // Updates the users state in the parent component with the fetched data. 
                // This will likely trigger a re-render of the parent and consequently the UserGrid 
                // with the new user data.
                setTransactions(data);
            
            } catch (error) { // If any error occurs during the try it logs the error to the console.
                console.error(error);
            
            // Always executes after the try or catch block. It sets isLoading to false, indicating 
            // that the data fetching attempt (whether successful or not) has completed.
            } finally {
                setIsLoading(false);
            }
        };
        getTransactions(); // Calls the getUsers function when the useEffect hook runs.
        }, 
        
        // Second argument is an array of dependencies. Tells React when to re-run the effect. 
        // The setUsers function from useState has a stable identity across renders and will never change. 
        // This means the effect will only run once when the component mounts
        [setTransactions]); 
    
    return (
        <>
            <VStack gap={0}>

                {transactions.map((transaction) => (
                    <TransactionCard key={transaction.id} transaction={transaction} setTransactions={setTransactions} />
                ))}
            
            </VStack>
        
            {isLoading && (
                <Flex justifyContent={"center"}>
                    <Spinner size={"xl"}/>
                </Flex>
            )}
            
            {!isLoading && transactions.length === 0 && (
                <Flex justifyContent={"center"}>
                    <Text textStyle="sm" fontWeight="light">No transactions found</Text>
                </Flex>
            )}
        
        </>
    );
};

export default TransactionGrid;
"""

In [26]:
file4 = """
// TransactionCard.jsx

import { Card, Flex, Avatar, Box, Heading, Text, IconButton, HStack } from '@chakra-ui/react'
import { Toaster, toaster } from "@/components/ui/toaster"

const TransactionCard = ({ transaction, setTransactions }) => {
        
    return (
        <Card.Root width="100%" bg="olive">
            <Toaster />
            <Card.Body gap="2">
                <Flex gap={"1"}>
                    
                    <HStack>
                            <Text>{transaction.id}</Text>
                            <Text>{transaction.date}</Text>
                            <Text>{transaction.description}</Text>
                            <Text>{transaction.value}</Text>
                            <Text>{transaction.childrenFlag}</Text>
                            <Text>{transaction.docFlag}</Text>
                    </HStack>


                    {/* <Flex flex={"1"} gap={"4"} alignItems={"center"}> 
                        <Box>
                            <Heading size='sm'>{transaction.value}</Heading>
                            <Text>{transaction.description}</Text>
                        </Box>
                    </Flex>
                
                    <Flex>
                        <EditModal usuario={usuario} setUsuarios={setUsuarios} />

                        <IconButton 
                            variant='ghost' 
                            colorPalette='red' 
                            size='sm' 
                            aria-label="See menu"
                            onClick={handleDeleteUser}
                        >
                            <BiTrash />
                        </IconButton>
                    </Flex> */}

                </Flex>
            </Card.Body>
            
            {/* <Card.Footer>
                <Text textStyle="sm" fontWeight="light">{transaction.id}</Text>
            </Card.Footer> */}

        </Card.Root>
    )
}

export default TransactionCard
"""

### Files outputed by the 1st LLM

In [59]:
path1 = Path(cwd.parent, 'frontend', 'src' ,'App.jsx')
path2 = Path(cwd.parent, 'frontend', 'src', 'components', 'ui' ,'Navbar.jsx')
path3 = Path(cwd.parent, 'frontend', 'src', 'components', 'ui' ,'TransactionGrid.jsx')
path4 = Path(cwd.parent, 'frontend', 'src', 'components', 'ui' ,'TransactionCard.jsx')
path_arr = [path1, path2, path3, path4]

files_presented = {}

for i, path in enumerate(path_arr):
    with open(path, 'r') as file:
        f = file.read()
    files_presented[i+1] = f

In [69]:
prompt = f"""
# 1. Context of the task:

An LLM was asked to improve the layout and color pallete of the React \
application using Chakra UI version 3. The layout was asked to be simple \
and minimalistic, and follow the structure and behavior presented \
in the files "App.jsx", "Navbar.jsx", "TransactionGrid.jsx" \
and "TransactionCard.jsx". 

## 1.1. Files presented to the LLM (delimited by triple backticks):

```{file1}```

```{file2}```

```{file3}```

```{file4}```

## 1.2. Eddited files with suggested changes by the LLM (delimited by triple backticks):

```
{files_presented[1]}
```

```
{files_presented[2]}
```

```
{files_presented[3]}
```

```
{files_presented[4]}
```

# 2. Instructions:

Your task is to evaluate the results presented by the LLM.\
Make brief comments about the suggested changes and suggest improvements, \
if it is the case, presenting the final files including your changes.

# 3. Chakra UI v3 Documentation for LLMs

Adicionaly, attention must be given to the Chakra UI v3 Documentation for LLMs \
available into the links presented below (delimited by triple quotes).

\"\"\"{chakra_docs}\"\"\"
"""

# Prompt final

In [72]:
print(prompt)


# 1. Context of the task:

An LLM was asked to improve the layout and color pallete of the React application using Chakra UI version 3. The layout was asked to be simple and minimalistic, and follow the structure and behavior presented in the files "App.jsx", "Navbar.jsx", "TransactionGrid.jsx" and "TransactionCard.jsx". 

## 1.1. Files presented to the LLM (delimited by triple backticks):

```
// App.jsx

import { Container, Stack, Text, Box } from "@chakra-ui/react"
import Navbar from "./components/ui/Navbar"
import TransactionGrid from "./components/ui/TransactionGrid"
import { useState } from "react"

export const BASE_URL = "http://127.0.0.1:5000/api"

function App() {

    const [transactions, setTransactions] = useState([])

    return (

            <Stack minH="100vh" gap={1}>
                <Navbar setUsers={setTransactions} />
                <Container maxW="100%" height="90vh" bg={"gray"}>
                    <Text 
                        fontSize={{ base: "3xl", md: "2